In [49]:
path = "C:\\Users\\noah\\repos\\BTLR\\wake\\data\\cv-corpus-15.0-delta-2023-09-08\\en\\clips\\common_voice_en_38024625.mp3"

In [50]:
import os
os.path.isfile(path)

True

In [51]:
import librosa
from parameters import MycroftParams as ap

y, sr = librosa.load(path, sr=ap.sample_rate)

In [52]:
type(y)

numpy.ndarray

In [53]:
import numpy as np
audio_data = np.squeeze(y)
audio_data = audio_data.astype(np.float32)

In [54]:
import data
mfcc = data.get_mfcc(audio_data)

In [55]:
'''
The first dimension of the MFCC is the number of individual frames (or windows)
that make up the audio
'''
from parameters import MycroftParams as ap
audio_length = len(audio_data)
print(audio_length)
window_size = ap.window_samples
hop_size = ap.hop_samples

number_frames = (audio_length - window_size) // hop_size + 1
print(number_frames)
print(len(mfcc))
assert(number_frames == len(mfcc))


82368
101
101


In [56]:
'''
The second dimension of the MFCC is always the coefficients, so the size will
be the number of spectrogram coefficients.
'''

assert(mfcc.shape[1] == ap.n_mfcc)

In [57]:
import numpy as np

# Load data
import librosa
y, sr = librosa.load(path, sr=None)
# Convert to numpy
import numpy as np
audio_data = np.squeeze(y)
audio_data = audio_data.astype(np.float32)
# Convert to MFCC
import data
mfcc = data.get_mfcc(audio_data)  # shape (total_timesteps x n_mfcc)
# Reshape to fit into model

# make sure it is even multiple of size n_features
remainder = len(mfcc) % ap.n_features

if (remainder != 0):
    # append zeros to beginning
    count_zeros = ap.n_features - remainder
    mfcc = np.concatenate([
            np.zeros((count_zeros, mfcc.shape[1])),
            mfcc
        ])
assert(len(mfcc) % ap.n_features == 0)

sections = len(mfcc) // ap.n_features
# input = np.split(mfcc, sections) # shape (n x n_features x n_mfcc)
input = np.reshape(mfcc, (-1, ap.n_features, ap.n_mfcc))

In [61]:
print(mfcc.shape)
print(input.shape)
index = 29
print(mfcc[index])
print(input[index // ap.n_features][index % ap.n_features])

def same(index):
    return np.array_equal(mfcc[index], input[index // ap.n_features][index % ap.n_features])

for i in range(len(mfcc)):
    if not same(i):
        print('not same')

(232, 13)
(8, 29, 13)
[-12.26137182   2.76322785   0.65285486   2.76426243   0.18142609
   1.91480465  -1.62400647   0.20190497  -1.58925434   1.37498852
  -1.41271307   0.05049408   0.04853711]
[-12.26137182   2.76322785   0.65285486   2.76426243   0.18142609
   1.91480465  -1.62400647   0.20190497  -1.58925434   1.37498852
  -1.41271307   0.05049408   0.04853711]
